In [47]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

# Spark Setup

In [48]:
# Step 1: Initialize SparkSession
# spark.stop()    
spark = SparkSession.builder \
    .appName("Read CSV Example") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()


# Step 2: Read CSV File
file_path = "./csv_file/scopus_data_2018_2023.csv"  # Replace with your file path
df1 = spark.read.csv(file_path, header=True, inferSchema=True)
file_path = "./csv_file/scopus_2024_1000_all_subjects_cleaned.csv"  # Replace with your file path
df2 = spark.read.csv(file_path, header=True, inferSchema=True)

# Step 3: Show the DataFrame
df1.show(5) 
df2.show(5)

+--------------------+--------------------+------------------+----+--------+--------------+-----------+
|        Source Title|                 DOI|               EID|Year|Abstract|Subjected Area|    Country|
+--------------------+--------------------+------------------+----+--------+--------------+-----------+
|Recoletos Multidi...|10.32871/rmrj1806...|2-s2.0-85166929946|2018|    NULL|          MULT|Philippines|
|Recoletos Multidi...|10.32871/rmrj1806...|2-s2.0-85160651147|2018|    NULL|          MULT|Philippines|
|Recoletos Multidi...|10.32871/rmrj1806...|2-s2.0-85160637511|2018|    NULL|          MULT|Philippines|
|Recoletos Multidi...|10.32871/rmrj1806...|2-s2.0-85160636983|2018|    NULL|          MULT|Philippines|
|Recoletos Multidi...|10.32871/rmrj1806...|2-s2.0-85153625255|2018|    NULL|          MULT|Philippines|
+--------------------+--------------------+------------------+----+--------+--------------+-----------+
only showing top 5 rows

+--------------------+-----------------

# Drop Unneccesary Columns

In [49]:
print("before " + str(df1.count()) + " " + str(len(df1.columns)))
df1 = df1.drop('Title', 'EID','Source Title','Abstract')

# Drop rows with non-null values below 50% of the total columns
thresh = int(0.5 * len(df1.columns))
df1 = df1.dropna(thresh=thresh)
df_clean = df1.dropna()

print("after " +str(df1.count()) + " " + str(len(df1.columns)))
df1.show(5)

print("before " + str(df2.count()) + " " + str(len(df2.columns)))
df2 = df2.drop('Title', 'EID','Source Title','Abstract')

# Drop rows with non-null values below 50% of the total columns
thresh = int(0.5 * len(df2.columns))
df2 = df2.dropna(thresh=thresh)
df_clean = df2.dropna()

print("after " +str(df2.count()) + " " + str(len(df2.columns)))
df2.show(5)

before 356451 7
after 356451 4
+--------------------+----+--------------+-----------+
|                 DOI|Year|Subjected Area|    Country|
+--------------------+----+--------------+-----------+
|10.32871/rmrj1806...|2018|          MULT|Philippines|
|10.32871/rmrj1806...|2018|          MULT|Philippines|
|10.32871/rmrj1806...|2018|          MULT|Philippines|
|10.32871/rmrj1806...|2018|          MULT|Philippines|
|10.32871/rmrj1806...|2018|          MULT|Philippines|
+--------------------+----+--------------+-----------+
only showing top 5 rows

before 41695 6
after 41695 4
+--------------------+----+--------------+--------------+
|                 DOI|Year|Subjected Area|       Country|
+--------------------+----+--------------+--------------+
|10.1016/j.isci.20...|2024|          MULT|        Canada|
|10.1016/j.isci.20...|2024|          MULT|         China|
|10.1016/j.isci.20...|2024|          MULT|         India|
|10.1016/j.isci.20...|2024|          MULT|United Kingdom|
|10.1016/j.isc

# Group Country,Subjected Area,Year

In [50]:
# print(df.select(df['EID']).distinct().count())

df_article = df1.groupBy('Country', 'Subjected Area', 'Year') \
    .agg(F.count('DOI').alias('Article Count'))

df_article_sorted = df_article.orderBy(F.desc('Article Count'))

# Show the aggregated data
df_article_sorted.show(5)

# Perform the join operation between df_article and df_clean
df1 = df1.join(df_article, 
                    on=['Country', 'Subjected Area', 'Year'], 
                    how='inner')

# Show the resulting DataFrame
# df_joined.show()
df1.show(5)

+-------+--------------+----+-------------+
|Country|Subjected Area|Year|Article Count|
+-------+--------------+----+-------------+
|  China|          ENER|2021|         1374|
|  China|          MATE|2021|         1331|
|  China|          MATE|2022|         1275|
|  China|          ENGI|2021|         1268|
|  China|          ENER|2022|         1238|
+-------+--------------+----+-------------+
only showing top 5 rows

+-------+--------------+----+-----------------+-------------+
|Country|Subjected Area|Year|              DOI|Article Count|
+-------+--------------+----+-----------------+-------------+
| Poland|          SOCI|2018|10.3390/w11010049|           21|
| Poland|          SOCI|2018|10.3390/w11010056|           21|
| Taiwan|          COMP|2019|             NULL|           20|
|  Spain|          ECON|2023|             NULL|           68|
|  Spain|          ECON|2023|             NULL|           68|
+-------+--------------+----+-----------------+-------------+
only showing top 5 ro

# Join Table

In [51]:
df_article = df2.groupBy('Country', 'Subjected Area', 'Year') \
    .agg(F.count('DOI').alias('Article Count'))

df_article_sorted = df_article.orderBy(F.desc('Article Count'))

# Show the aggregated data
df_article_sorted.show(5)
# Perform the join operation between df_article and df_clean
df2 = df2.join(df_article, 
                    on=['Country', 'Subjected Area', 'Year'], 
                    how='inner')

# Show the resulting DataFrame
# df_joined.show()
df2.show(5)


+-------+--------------+----+-------------+
|Country|Subjected Area|Year|Article Count|
+-------+--------------+----+-------------+
|  China|          CHEM|2024|         1042|
|  China|          CENG|2024|         1038|
|  China|          ENGI|2024|          987|
|  China|          MATE|2024|          921|
|  China|          ENER|2024|          906|
+-------+--------------+----+-------------+
only showing top 5 rows

+--------------+--------------+----+--------------------+-------------+
|       Country|Subjected Area|Year|                 DOI|Article Count|
+--------------+--------------+----+--------------------+-------------+
|        Canada|          MULT|2024|10.1016/j.isci.20...|           21|
|         China|          MULT|2024|10.1016/j.isci.20...|          493|
|         India|          MULT|2024|10.1016/j.isci.20...|           91|
|United Kingdom|          MULT|2024|10.1016/j.isci.20...|           29|
|   Netherlands|          MULT|2024|10.1016/j.isci.20...|           17|
+--

# Rename Columns

In [52]:
df1 = df1.select(
    df1['DOI'].alias('doi'),
    df1['Year'].alias('year'),
    df1['Subjected Area'].alias('subject_area_abbrev'),
    df1['Country'].alias('country'),
    df1['Article Count'].alias('article_amount')
)

# Display the transformed DataFrame
df1.show(5)

df2 = df2.select(
    df2['DOI'].alias('doi'),
    df2['Year'].alias('year'),
    df2['Subjected Area'].alias('subject_area_abbrev'),
    df2['Country'].alias('country'),
    df2['Article Count'].alias('article_amount')
)

# Display the transformed DataFrame
df2.show(5)

+-----------------+----+-------------------+-------+--------------+
|              doi|year|subject_area_abbrev|country|article_amount|
+-----------------+----+-------------------+-------+--------------+
|10.3390/w11010049|2018|               SOCI| Poland|            21|
|10.3390/w11010056|2018|               SOCI| Poland|            21|
|             NULL|2019|               COMP| Taiwan|            20|
|             NULL|2023|               ECON|  Spain|            68|
|             NULL|2023|               ECON|  Spain|            68|
+-----------------+----+-------------------+-------+--------------+
only showing top 5 rows

+--------------------+----+-------------------+--------------+--------------+
|                 doi|year|subject_area_abbrev|       country|article_amount|
+--------------------+----+-------------------+--------------+--------------+
|10.1016/j.isci.20...|2024|               MULT|        Canada|            21|
|10.1016/j.isci.20...|2024|               MULT|    

# New Order

In [53]:
new_column_order = ['year', 'country','article_amount', 'subject_area_abbrev']

# Reorder the DataFrame
df1 = df1.select(*new_column_order)
df1.show(5)

df2 = df2.select(*new_column_order)
df2.show(5)

+----+-------+--------------+-------------------+
|year|country|article_amount|subject_area_abbrev|
+----+-------+--------------+-------------------+
|2018| Poland|            21|               SOCI|
|2018| Poland|            21|               SOCI|
|2019| Taiwan|            20|               COMP|
|2023|  Spain|            68|               ECON|
|2023|  Spain|            68|               ECON|
+----+-------+--------------+-------------------+
only showing top 5 rows

+----+-------+--------------+-------------------+
|year|country|article_amount|subject_area_abbrev|
+----+-------+--------------+-------------------+
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
+----+-------+--------------+-------------------+
only showing top 5 rows



# Impute Missing Value

In [54]:
country_mode_value = df1.groupBy('country').count().orderBy(F.desc('count')).first()[0]
print(country_mode_value)

subject_mode_value = df1.groupBy('subject_area_abbrev').count().orderBy(F.desc('count')).first()[0]
print(subject_mode_value)

article_mean_value = df1.agg(F.avg('article_amount').alias('mean_article_amount')).collect()[0]['mean_article_amount']
print(article_mean_value)

df = df1.na.fill(
    {
        'country' : country_mode_value, 
        'subject_area_abbrev': subject_mode_value,
        'article_amount': article_mean_value,
    
    })

country_mode_value = df2.groupBy('country').count().orderBy(F.desc('count')).first()[0]
print(country_mode_value)

subject_mode_value = df2.groupBy('subject_area_abbrev').count().orderBy(F.desc('count')).first()[0]
print(subject_mode_value)

article_mean_value = df2.agg(F.avg('article_amount').alias('mean_article_amount')).collect()[0]['mean_article_amount']
print(article_mean_value)

df = df2.na.fill(
    {
        'country' : country_mode_value, 
        'subject_area_abbrev': subject_mode_value,
        'article_amount': article_mean_value,
    
    })

China
NEUR
297.8949403242188
China
NEUR
333.16440820242235


In [55]:
# Count unique values for each column
print("2018-2023 : ")
for column in df1.columns:
    unique_count = df1.select(column).distinct().count()
    print(f"Unique values in column '{column}': {unique_count}")

print("2024 : ")
for column in df2.columns:
    unique_count = df2.select(column).distinct().count()
    print(f"Unique values in column '{column}': {unique_count}")

2018-2023 : 
Unique values in column 'year': 6
Unique values in column 'country': 200
Unique values in column 'article_amount': 419
Unique values in column 'subject_area_abbrev': 27
2024 : 
Unique values in column 'year': 1
Unique values in column 'country': 155
Unique values in column 'article_amount': 133
Unique values in column 'subject_area_abbrev': 27


In [56]:
from pyspark.sql import SparkSession


file_path_new = r"C:\Users\admin\Downloads\Data 2018-2023\Project\csv_file\2018_2023_data_prep_starter.csv"

df_new = spark.read.csv(file_path_new, header=True, inferSchema=True)

df2 = df2.union(df_new)

df2.show(5)


+----+-------+--------------+-------------------+
|year|country|article_amount|subject_area_abbrev|
+----+-------+--------------+-------------------+
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
|2024|   Iran|            50|               AGRI|
+----+-------+--------------+-------------------+
only showing top 5 rows



# Remove remove outlier and Save Data

In [57]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

def remove_outliers_left(df, col):
    # Convert the column to DoubleType
    df = df.withColumn(col, F.col(col).cast(DoubleType()))
    
    # Calculate the 1st percentile (1%) of the specified column
    percentile_1 = df.approxQuantile(col, [0.20], 0.0)[0]
    print(f"1st Percentile for {col}: {percentile_1}")
    
    # Filter out values less than the 1st percentile
    # return df.filter(F.col(col) >= 50)
    return df.filter(F.col(col) >= percentile_1)

# Apply the function to remove left-side outliers (below 1st percentile)
df1 = remove_outliers_left(df1, "article_amount")
df2 = remove_outliers_left(df2, "article_amount")


1st Percentile for article_amount: 25.0
1st Percentile for article_amount: 18.0


In [58]:
import os

# Folder where the CSV files will be saved
folder_path = '/csv_file/'  # Specify the folder path where you want to save the CSV files

# Ensure the folder exists (create if it doesn't)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Convert the DataFrames to Pandas
pandas_df1 = df1.toPandas()
pandas_df2 = df2.toPandas()

# Export to CSV files in the specified folder
pandas_df1.to_csv(os.path.join(folder_path, 'final_data_prep_2018-2023.csv'), index=False)
pandas_df2.to_csv(os.path.join(folder_path, 'final_data_prep_2024.csv'), index=False)

print("CSV files have been exported successfully.")


CSV files have been exported successfully.


In [59]:
import pandas as pd
import os

# Define file paths
file1 = "./csv_file/final_data_prep_2018-2023.csv"
file2 = "./csv_file/2018_2023_data_prep_starter.csv"

# Read the CSV files
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Combine the DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Define the output folder and file path
folder_path = '/csv_file/'  # Specify the folder path where the combined file will be saved
output_file = os.path.join(folder_path, 'combined_data_2018_2023.csv')

# Ensure the folder exists (create if it doesn't)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Export the combined data to CSV
combined_df.to_csv(output_file, index=False)

print(f"Combined data saved to {output_file}")


Combined data saved to /csv_file/combined_data_2018_2023.csv
